In [1]:
from tensorflow.keras.regularizers import l2
from keras_tuner import HyperModel, HyperParameters, RandomSearch

from utils.util import is_laptop
from utils.model import mean_absolute_percentage_error
from utils.data import get_normalized_data

import pandas as pd, numpy as np, tensorflow as tf

In [2]:
train, val, test = get_normalized_data()

X_train, y_train = train
X_val, y_val = val
X_test, y_test = test

In [3]:
hp = HyperParameters()
hp.Int('n_layers', min_value = 4, max_value = 15)
hp.Int('n_units', min_value = 32, max_value = 512, step = 32)
hp.Int('epochs', min_value = 50, max_value = 1000, step = 50)

hp.Float('penalty', min_value = 1e-4, max_value = 1, sampling = 'log')

hp.Choice('activation', ['relu', 'elu', 'softmax'])
hp.Choice('optimizer', ['adam', 'rmsprop', 'adamax'])


'adam'

In [4]:
def build_model_tuner(hp):
    n_units = hp.get('n_units')
    n_layers = hp.get('n_layers')
    activation = hp.get('activation')
    penalty = hp.get('penalty')
    optimizer = hp.get('optimizer')

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units = n_units, activation = activation, kernel_regularizer=l2(penalty), input_shape = (X_train.shape[1], )))
    model.add(tf.keras.layers.BatchNormalization())

    for _ in range(n_layers):
        model.add(tf.keras.layers.Dense(units = n_units, activation = activation, kernel_regularizer=l2(penalty)))
        model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Dense(units = n_units, activation = activation, kernel_regularizer=l2(penalty)))
    model.add(tf.keras.layers.Dropout(0.5))

    model.add(tf.keras.layers.Dense(units = 1, activation = 'linear'))

    model.compile(optimizer = optimizer, loss = mean_absolute_percentage_error)

    return model



In [5]:
tuner = RandomSearch(build_model_tuner, hyperparameters = hp, objective = 'val_loss', max_trials = 10)
tuner.search(X_train, y_train, epochs = hp.get('epochs'), validation_data = (X_val, y_val))

Trial 10 Complete [00h 06m 07s]
val_loss: 5.034312725067139

Best val_loss So Far: 5.0051984786987305
Total elapsed time: 02h 27m 08s


In [6]:
from datetime import date
best_model = tuner.get_best_models()[0]
best_model.save(f'./saved_models/nn-{date.today()}.keras')